## CV02 - ResNet Ablation Study

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import urllib3
urllib3.disable_warnings()

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [4]:
print(ds_info.features)

FeaturesDict({
    'id': Text(shape=(), dtype=tf.string),
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [5]:
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(50000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)


In [6]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [15]:
# Residual 블록 구현

def residual_block(input_layer, channel):
    
    # shoutcuts을 하면 입력 레이어와 출력 레이어의 채널 수가 달라지므로 맞춰준다.
    if input_layer.shape[-1] != channel:
        inchange_layer = Conv2D(channel, (1, 1), padding='valid', activation='relu',
                             kernel_initializer='he_normal')(input_layer)
    BatchNormalization()    
    conv1 = Conv2D(channel, (3, 3), padding='same', activation='relu',
                   kernel_initializer='he_normal')(input_layer)
    conv2 = Conv2D(channel, (3, 3), padding='same', activation='linear',
                   kernel_initializer='he_normal')(conv1)
    
    output = add([conv2, inchange_layer])
    output = Activation('relu')(output)
    return output

In [16]:
resnet_input_layer = Input(shape=(32, 32, 3))
resnet_block_output = residual_block(resnet_input_layer, 64)

model = keras.Model(inputs=resnet_input_layer, outputs=resnet_block_output)  

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 32, 32, 64)   1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 32, 32, 64)   36928       conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 32, 64)   256         input_6[0][0]                    
____________________________________________________________________________________________

In [15]:
# ResNet 기본 블록 구현

def resnet_block(input_layer, channel, is_50, is_plain, repetition, stage):
    
    x = input_layer
    
    if is_50 is False:
        for i in range(repetition):
            init_strides=(1, 1)
            if stage > 0 and i == 0:
                init_strides=(2, 2)
            x = Conv2D(filters=channel, kernel_size=(3, 3), strides=init_strides,
                       padding='same', kernel_initialzer='he_normal'),
            
                        
                        
            

    if num_cnn == 2:
        for cnn_num in range(num_cnn):
            x = Conv2D(filters=channel, kernel_size=(3, 3), activation='relu',
                            kernel_initializer='he_normal', padding='same',
                            name=f'block{block_num}_conv{cnn_num}')(x)
    else:
        x = Conv2D(filters=channel, kernel_size=(1, 1), activation='relu',
                        kernel_initializer='he_normal', padding='same',
                        name=f'block{block_num}_conv1')(x)
        x = Conv2D(filters=channel, kernel_size=(3, 3), activation='relu',
                        kernel_initializer='he_normal', padding='same',
                        name=f'block{block_num}_conv2')(x)
        x = Conv2D(filters=channel*2, kernel_size=(1, 1), activation='relu',
                        kernel_initializer='he_normal', padding='same',
                        name=f'block{block_num}_conv3')(x)
        
    x = MaxPooling2D(pool_size=(2, 2), strides=2,
                     name=f'block{block_num}_pooling')(x)
    
    return x

In [16]:
resnet_input_layer = Input(shape=(32, 32, 3))
resnet_block_output = build_resnet_block(resnet_input_layer)

In [17]:
# 블록 1개짜리 model 생성
model = keras.Model(inputs=resnet_input_layer, outputs=resnet_block_output)  

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv0 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pooling (MaxPooling2D (None, 16, 16, 64)        0         
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 블록을 조합하여 resnet-34 모델 만들기

def build_resnet(input_shape=(32, 32, 3),
                num_cnn=2,
                channel_list=[64, 128, 256, 512],
                num_classes=10):
    
    input_layer = Input(shape=input_shape)
    output = input_layer
    
    for i, channel in enumerate(channel_list):
        output = build_resnet_block(output, num_cnn=num_cnn, channel=channel, block_num=i)
        
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(4096, activation='relu', name='fc1')(output)
    output = keras.layers.Dense(4096, activation='relu', name='fc2')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)